In [1]:
# Mount your Google drive folder on Colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import pandas as pd
import nltk
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
#from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perform cosine similarity
from nltk import word_tokenize #  to create tokens
#from nltk.corpus import stopwords # for stop words

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
df = pd.read_csv('/content/gdrive/My Drive/wine_tasting_dataprep.csv')

In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
variety_list = list(df['variety'].unique())
winery_list = list(df['winery'].unique())
location_list = list(df['Location'].unique())

In [9]:
print('variety', len(variety_list))
print('winery', len(winery_list))
print('location', len(location_list))

variety 750
winery 19176
location 1783


In [10]:
def VID(text):
  for i in range(len(variety_list)):
    if text == variety_list[i]:
      return i
def WID(text):
  for i in range(len(winery_list)):
    if text == winery_list[i]:
      return i
def LID(text):
  for i in range(len(location_list)):
    if text == location_list[i]:
      return i

In [11]:
df['variety_ID'] = df['variety'].apply(VID)
df['winery_ID'] = df['winery'].apply(WID)
df['location_ID'] =df['Location'].apply(LID)

In [12]:
df.head()

,description,variety,winery,Location,lemmatized_text,variety_ID,winery_ID,location_ID
0,This tremendous 100% varietal wine hails from ...,Cabernet Sauvignon,Heitz,Napa Valley California US,this tremendous varietal wine hail from oakvil...,0,0,0
1,"Ripe aromas of fig, blackberry and cassis are ...",Tinta de Toro,Bodega Carmen Rodríguez,Toro Northern Spain,ripe aroma of fig blackberry and cassis be sof...,1,1,1
2,Mac Watson honors the memory of a wine once ma...,Sauvignon Blanc,Macauley,Knights Valley Sonoma California US,mac watson honor the memory of a wine once mak...,2,2,2
3,"This spent 20 months in 30% new French oak, an...",Pinot Noir,Ponzi,Willamette Valley Oregon US,this spent month in new french oak and incorpo...,3,3,3
4,"This is the top wine from La Bégude, named aft...",Provence red blend,Domaine de la Bégude,Bandol Provence France,this be the top wine from la b gude name after...,4,4,4


In [13]:
# using tf-idf
 
tfidf = TfidfVectorizer(max_features=684,min_df=7, max_df=0.8, stop_words=stopwords.words('english')) # initializing tf-idf
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray() # transforming the data to array

In [14]:
variety_id = np.array(list(df['variety_ID']))
winery_id = np.array(list(df['winery_ID']))
location_id = np.array(list(df['location_ID']))

In [15]:
from sklearn.model_selection import train_test_split
Xv_train, Xv_test, yv_train, yv_test = train_test_split(x_tfidf, variety_id, test_size=0.2, random_state=101)
Xw_train, Xw_test, yw_train, yw_test = train_test_split(x_tfidf, winery_id, test_size=0.2, random_state=101)
Xl_train, Xl_test, yl_train, yl_test = train_test_split(x_tfidf, location_id, test_size=0.2, random_state=101)

In [16]:
x_tfidf.shape

(410744, 684)

In [17]:
# creating a model for predicting variety
model_var  = tf.keras.Sequential([
                tf.keras.layers.Flatten(input_shape=(1,684)),
                tf.keras.layers.Dense(24, activation = 'relu'),
                tf.keras.layers.Dense(750, activation = 'softmax')
])

model_var.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics =['accuracy'])


num_epochs = 150

history = model_var.fit(Xv_train, yv_train, epochs = num_epochs,
                    validation_data = (Xv_test,yv_test))

Epoch 1/150
10269/10269 [==============================] - 26s 3ms/step - loss: 2.5240 - accuracy: 0.4235 - val_loss: 2.0142 - val_accuracy: 0.5156
Epoch 2/150
10269/10269 [==============================] - 26s 3ms/step - loss: 1.9206 - accuracy: 0.5264 - val_loss: 1.8688 - val_accuracy: 0.5338
Epoch 3/150
10269/10269 [==============================] - 24s 2ms/step - loss: 1.8169 - accuracy: 0.5422 - val_loss: 1.8102 - val_accuracy: 0.5442
Epoch 4/150
10269/10269 [==============================] - 25s 2ms/step - loss: 1.7651 - accuracy: 0.5488 - val_loss: 1.7796 - val_accuracy: 0.5483
Epoch 5/150
10269/10269 [==============================] - 25s 2ms/step - loss: 1.7344 - accuracy: 0.5538 - val_loss: 1.7622 - val_accuracy: 0.5500
Epoch 6/150
10269/10269 [==============================] - 25s 2ms/step - loss: 1.7134 - accuracy: 0.5566 - val_loss: 1.7499 - val_accuracy: 0.5517
Epoch 7/150
10269/10269 [==============================] - 24s 2ms/step - loss: 1.6984 - accuracy: 0.5584 - val_